In [1]:
import torch
from torch.autograd import Variable
from torchvision import models
import cv2
import sys
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import dataset
from prune import *
from torchsummary import summary
import argparse
from operator import itemgetter
from heapq import nsmallest
import time

In [4]:
class ModifiedVGG16Model(torch.nn.Module):
    def __init__(self):
        super(ModifiedVGG16Model, self).__init__()

        model = models.vgg16(pretrained=True)
        self.features = model.features

        for param in self.features.parameters():
            param.requires_grad = False

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(25088, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2))

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
model = torch.load("model_prunned_half_256.pkl")
model.cuda()

ModifiedVGG16Model(
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=5439, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_features=4096, bias=True)
    (5): ReLU(inplace)
    (6): Linear(in_features=4096, out_features=2, bias=True)
  )
  (features): Sequential(
    (0): Conv2d(3, 47, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(47, 52, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(52, 101, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(101, 102, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(102, 184, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [5]:
summary(model,(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 47, 224, 224]           1,316
              ReLU-2         [-1, 47, 224, 224]               0
            Conv2d-3         [-1, 52, 224, 224]          22,048
              ReLU-4         [-1, 52, 224, 224]               0
         MaxPool2d-5         [-1, 52, 112, 112]               0
            Conv2d-6        [-1, 101, 112, 112]          47,369
              ReLU-7        [-1, 101, 112, 112]               0
            Conv2d-8        [-1, 102, 112, 112]          92,820
              ReLU-9        [-1, 102, 112, 112]               0
        MaxPool2d-10          [-1, 102, 56, 56]               0
           Conv2d-11          [-1, 184, 56, 56]         169,096
             ReLU-12          [-1, 184, 56, 56]               0
           Conv2d-13          [-1, 177, 56, 56]         293,289
             ReLU-14          [-1, 177,